CS551g Data Mining and Visualisation <br>
Practical 3 - Text Summarisation <br>

**STEP 1: Familiarize yourself with the code below**<br>
The code builds the p(word) model in getProbabilities()


*Use the "files" view on the left navigation bar to upload nokia-neg.txt and nokia-pos.txt (from NokiaTxt(Part1).zip) before runing the code below

In [ ]:
import re, random, math, collections, itertools

PRINT_ERRORS=0
#calculates p(W) 
def getProbabilities(sentences, pWord):
    freq = {} # {} initialises a dictionary [hash function]
    allWordsTot = 0
    #iterate through each sentence
    for sentence in sentences:
        wordList = re.findall(r"[\w']+", sentence)

        for word in wordList: #iterate over words in sentence 
            allWordsTot += 1 # keeps count of total words in dataset
            if not (word in freq):
                freq[word] = 1
            else:    
                freq[word] += 1


        # Calculate p(word)
    for word in freq.keys(): 
        pWord[word] = (freq[word] / float(allWordsTot) )

#1. initialise datasets 
sentencesPos={};
sentencesNeg={};

file1 = "nokia-pos.txt"   #change this for other datasets
file2 = "nokia-neg.txt"   #change this for other datasets

F = open(file1, 'r', encoding="ISO-8859-1")
sentencesPos = re.split(r'\n', F.read())

F = open(file2, 'r', encoding="ISO-8859-1")
sentencesNeg = re.split(r'\n', F.read())

pWordPos={}    # p(W) 
pWordNeg={}    # p(W) 

#build probabilities using one of the files
getProbabilities(sentencesPos,  pWordPos)
getProbabilities(sentencesNeg,  pWordNeg)



#query-based summary? 
#increase probability of some words in query

query=[]
#query=["screen"] 

for word in query:
    if (word in pWordPos):
        pWordPos[word] *=100
    if (word in pWordNeg):
        pWordNeg[word] *=100

 **STEP 2: Produce summaries 
using scoreSentences()**<br>
This function iteratively selects the highest score
sentences till the word limit is reached<br>

In [ ]:
#summarise by selecting the sentences with the highest score
def scoreSentences(sentences, pWord, maxLength):
    freq = {} # {} initialises a dictionary [hash function]
    allWordsTot = 0
    scores = {}
    lengths = {} 
    #iterate through each sentence
    for sentence in sentences:
        wordList = re.findall(r"[\w']+", sentence)
        length=1
        score=0
        for word in wordList: #iterate over words
            length += 1
            score += pWord[word]
        scores[sentence] = score / length #calculate sentence score
        if length <= 3: #ignore short sentences
            scores[sentence] = 0
        lengths[sentence] = length #calculate sentence length


        #generate summary by incrementally including sentences in orde of importance
    summaryLength=0

    for s in sorted(scores, key=scores.get, reverse=True): #sort sentences by score
        
        if summaryLength + lengths[s] <=maxLength : #keep to word limit, and ignore short sentences
            print ("[%0.4f]" % scores[s], s) # print sentence
            summaryLength += lengths[s] #increment summary length

    print ("length = ", summaryLength, " words") 
    print ("")

print ("Positive Summary")
scoreSentences(sentencesPos, pWordPos, 100)

print ("Negative Summary")
scoreSentences(sentencesNeg, pWordNeg, 100)

**STEP 3: Produce summaries using scoreSentences2()**<br>

This function iteratively selects the highest score
sentences and updates the probabilits of words in that sentence (to prevent
repetition of information) untill the word limit is reached<br><br>


Are the results different? Which is better?

In [ ]:
#summarise by selecting the sentences with the highest score
def scoreSentences2(sentences, pWord_original, maxLength):
    pWord = pWord_original.copy()
    freq = {} # {} initialises a dictionary [hash function]
    allWordsTot = 0
    scores = {}
    lengths = {} 

    summaryLength=0
    #main loop
    iters=0;
    while summaryLength<=maxLength and iters<50:
        iters += 1
        #recalculate scores each time
        for sentence in sentences:
            wordList = re.findall(r"[\w']+", sentence)
            length=1
            score=0
            for word in wordList: #iterate over words
                length += 1
                score += pWord[word]
                
            scores[sentence] = score / length #calculate sentence score
            if length <= 3: #ignore short sentences
                scores[sentence] = 0
            lengths[sentence] = length #calculate sentence length


        #generate summary by including best sentences and updating probabilities 
        s= max(scores, key=scores.get)
        
        if summaryLength  <=maxLength: #keep to word limit
            if summaryLength + lengths[s]<=maxLength:
                print ("[%0.4f]" % scores[s], s) # print sentence
                summaryLength += lengths[s] #increment summary length
            wordList = re.findall(r"[\w']+", s)
            for word in wordList: #iterate over words and reduce probabilities
                pWord[word] *= pWord[word]
            

    print ("length = ", summaryLength, " words") 
    print ("")

print ("Positive Summary")
scoreSentences2(sentencesPos, pWordPos, 100)

print ("Negative Summary")
scoreSentences2(sentencesNeg, pWordNeg, 100)




In [ ]:
#Print out n most useful predictors
def mostUseful(pWord, n):
    print ("Highest Probability Words:")
    i=0
    for w in sorted(pWord, key=pWord.get, reverse=True):
        i += 1
        if i<n:
            print (w, "%0.4f" % pWord[w])

#    print ("\nLowest Probability Words:")
 #   i=0
  #  for w in sorted(pWord, key=pWord.get, reverse=False):
  #      i += 1
   #     if i<n:
    #        print (w, "%0.4f" % pWord[w])


# print most useful words
mostUseful(pWordPos, 50)

**STEP 4: Look for the commented-out line: #query=["screen"] in the first block of code and uncomment it**<br>
All this code does is define a list of words the user is interested in (currently
populated with "screen". The following lines increase the probabilities of the words
in this list to increase the chance of their selection in the summary. <br>
Think of some aspects to do with mobile phones and modify the query and look at
the summaries with both summarisers.<br>
Can you see a problem for scoreSentences2() with the query-based task? <br>